# Contents
* [data](#data)
* [transfer](#transfer)
* [eval](#eval)

In [1]:
from __future__ import print_function, division

import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision

from glob import glob
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from torch.autograd import Variable
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

plt.ion() 

### data

In [2]:
class MarketDataset(Dataset):
    def __init__(self, df, le=None, transform=None):
        self.transform = transform
        self.img_paths = np.array(df['pth'])
        
        if le is not None:
            self.has_classes = True
            self.classes = np.array(df['cl'].drop_duplicates())
            self.labels = torch.from_numpy(le.transform((df['cl'])))
        else:
            self.has_classes = False

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, index):
        img_path = self.img_paths[index]
        img = Image.open(img_path)

        if self.transform:
            img = self.transform(img)

        if self.has_classes:
            cl = self.labels[index]
            return (img, cl)
        else:
            return img

In [3]:
def create_df(x):
    df = pd.DataFrame({'pth': x})
    df['cl'] = df.pth.apply(lambda x: int(x.split('/')[-1].split('_')[0]))
    return df

In [4]:
root = 'data/Market-1501-v15.09.15'
paths = {
    'train': 'bounding_box_train/*.jpg',
    #'test': 'bounding_box_test/*.jpg',
    #'query': 'query/*.jpg'
}

dfs = {
    x: create_df(glob(os.path.join(root, paths[x])))for x in paths.keys()
}

for key in dfs.keys():
    print(key, len(dfs[key]))
    dfs[key].to_csv('%s.csv' % key, index=None, header=None)

train 12936


In [6]:
train_cl = np.array([int(x.split('/')[-1].split('_')[0]) for x in dfs['train']['pth']])
train_tr, val_tr, train_tr_cl, val_tr_cl = train_test_split(dfs['train']['pth'], train_cl, test_size=0.1, random_state=18, stratify=train_cl)
np.unique(train_tr_cl).shape, np.unique(val_tr_cl).shape

((751,), (693,))

In [7]:
dfs['train_tr'] = create_df(train_tr)
dfs['val_tr'] = create_df(val_tr)

In [8]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Scale([256, 256]),
        transforms.RandomCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Scale([256, 256]),
        transforms.CenterCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

In [9]:
le = preprocessing.LabelEncoder()
le.fit(train_cl)

LabelEncoder()

In [10]:
transform_dict = {
    'train_tr': 'train',
    'val_tr': 'test'
}

image_datasets = {
    x: MarketDataset(dfs[x], le=le, transform=data_transforms[transform_dict[x]]) 
    for x in ['train_tr', 'val_tr']
}

### transfer

In [11]:
dataloders = {
    x[0]: torch.utils.data.DataLoader(image_datasets[x[0]], batch_size=x[1],
                                      shuffle=True, num_workers=4)
    for x in [('train_tr', 64), ('val_tr', 4)]
}
              
dataset_sizes = {x: len(image_datasets[x]) for x in ['train_tr', 'val_tr']}
class_names = image_datasets['train_tr'].classes

use_gpu = torch.cuda.is_available()

In [12]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        for phase in ['train_tr', 'val_tr']:
            if phase == 'train_tr':
                scheduler.step()
                model.train(True)
            else:
                model.train(False)

            running_loss = 0.0
            running_corrects = 0

            for data in dataloders[phase]:
                inputs, labels = data

                if use_gpu:
                    inputs = Variable(inputs.cuda())
                    labels = Variable(labels.cuda())
                else:
                    inputs, labels = Variable(inputs), Variable(labels)

                optimizer.zero_grad()

                outputs = model(inputs)
                _, preds = torch.max(outputs.data, 1)
                loss = criterion(outputs, labels)

                if phase == 'train_tr':
                    loss.backward()
                    optimizer.step()

                running_loss += loss.data[0]
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            if phase == 'val_tr' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    #model.load_state_dict(best_model_wts)
    return model

In [19]:
model_conv = models.resnet50(pretrained=True)

for param in model_conv.parameters():
    param.requires_grad = False
    
num_ftrs = model_conv.fc.in_features
model_conv.fc = torch.nn.Sequential()
model_conv.fc.add_module('dropout', torch.nn.Dropout(p=0.75))
model_conv.fc.add_module('fc1', nn.Linear(num_ftrs, 751))

if use_gpu:
    model_conv = model_conv.cuda()
    
model_conv

ResNet (
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  (relu): ReLU (inplace)
  (maxpool): MaxPool2d (size=(3, 3), stride=(2, 2), padding=(1, 1), dilation=(1, 1))
  (layer1): Sequential (
    (0): Bottleneck (
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU (inplace)
      (downsample): Sequential (
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
      )
    )
    (1): Bott

In [48]:
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(model_conv.fc.parameters(), lr=0.1, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [49]:
model_conv = train_model(model_conv, criterion, optimizer_ft,
                         exp_lr_scheduler, num_epochs=15)

Epoch 0/14
----------
train_tr Loss: 0.1549 Acc: 0.0176
val_tr Loss: 0.1372 Acc: 0.0603

Epoch 1/14
----------
train_tr Loss: 0.1858 Acc: 0.0459
val_tr Loss: 0.1311 Acc: 0.1028

Epoch 2/14
----------
train_tr Loss: 0.2028 Acc: 0.0581
val_tr Loss: 0.1399 Acc: 0.1128

Epoch 3/14
----------
train_tr Loss: 0.2221 Acc: 0.0708
val_tr Loss: 0.1389 Acc: 0.1461

Epoch 4/14
----------
train_tr Loss: 0.2255 Acc: 0.0858
val_tr Loss: 0.1384 Acc: 0.1816

Epoch 5/14
----------
train_tr Loss: 0.2355 Acc: 0.0929
val_tr Loss: 0.1454 Acc: 0.1685

Epoch 6/14
----------
train_tr Loss: 0.2461 Acc: 0.0949
val_tr Loss: 0.1449 Acc: 0.1971

Epoch 7/14
----------
train_tr Loss: 0.2062 Acc: 0.1312
val_tr Loss: 0.0834 Acc: 0.3138

Epoch 8/14
----------
train_tr Loss: 0.1879 Acc: 0.1449
val_tr Loss: 0.0790 Acc: 0.3207

Epoch 9/14
----------
train_tr Loss: 0.1873 Acc: 0.1438
val_tr Loss: 0.0779 Acc: 0.3369

Epoch 10/14
----------
train_tr Loss: 0.1829 Acc: 0.1457
val_tr Loss: 0.0757 Acc: 0.3400

Epoch 11/14
--------

In [13]:
#torch.save(model_conv, 'finetuned_resnet_market_fc_lr1s7g1.pt')
model_conv = torch.load('finetuned_resnet_market_fc_lr1s7g1.pt')

In [20]:
for param in model_conv.parameters():
    param.requires_grad = True

In [21]:
params_dict = dict(model_conv.named_parameters())
params = []
for key, value in params_dict.items():
    print(key)
    if ('conv1' in key) & (not 'layer' in key):
        params += [{'params':[value],'lr':0.00001}]
    elif ('conv' in key):# | ('fc1' in key):
        params += [{'params':[value],'lr':0.002}]
    elif ('fc1' in key):
        params += [{'params':[value],'lr':0.1}]
    elif 'bias' in key:
        params += [{'params':[value], 'lr':0.0001, 'weight_decay':0}]
    else:
        params += [{'params':[value], 'lr':0.0001}] #'weight_decay':0.0001

conv1.weight
bn1.weight
bn1.bias
layer1.0.conv1.weight
layer1.0.bn1.weight
layer1.0.bn1.bias
layer1.0.conv2.weight
layer1.0.bn2.weight
layer1.0.bn2.bias
layer1.0.conv3.weight
layer1.0.bn3.weight
layer1.0.bn3.bias
layer1.0.downsample.0.weight
layer1.0.downsample.1.weight
layer1.0.downsample.1.bias
layer1.1.conv1.weight
layer1.1.bn1.weight
layer1.1.bn1.bias
layer1.1.conv2.weight
layer1.1.bn2.weight
layer1.1.bn2.bias
layer1.1.conv3.weight
layer1.1.bn3.weight
layer1.1.bn3.bias
layer1.2.conv1.weight
layer1.2.bn1.weight
layer1.2.bn1.bias
layer1.2.conv2.weight
layer1.2.bn2.weight
layer1.2.bn2.bias
layer1.2.conv3.weight
layer1.2.bn3.weight
layer1.2.bn3.bias
layer2.0.conv1.weight
layer2.0.bn1.weight
layer2.0.bn1.bias
layer2.0.conv2.weight
layer2.0.bn2.weight
layer2.0.bn2.bias
layer2.0.conv3.weight
layer2.0.bn3.weight
layer2.0.bn3.bias
layer2.0.downsample.0.weight
layer2.0.downsample.1.weight
layer2.0.downsample.1.bias
layer2.1.conv1.weight
layer2.1.bn1.weight
layer2.1.bn1.bias
layer2.1.conv2.we

In [22]:
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(params, momentum=0.9, weight_decay=0.0001)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=40, gamma=0.1)

In [ ]:
# without fc finetuning

In [23]:
model_conv = train_model(model_conv, criterion, optimizer_ft,
                         exp_lr_scheduler, num_epochs=50)

Epoch 0/49
----------
train_tr Loss: 0.1031 Acc: 0.0489
val_tr Loss: 1.0796 Acc: 0.2179

Epoch 1/49
----------
train_tr Loss: 0.0623 Acc: 0.2287
val_tr Loss: 0.6090 Acc: 0.4745

Epoch 2/49
----------
train_tr Loss: 0.0419 Acc: 0.4157
val_tr Loss: 0.3895 Acc: 0.6321

Epoch 3/49
----------
train_tr Loss: 0.0305 Acc: 0.5404
val_tr Loss: 0.2844 Acc: 0.7272

Epoch 4/49
----------
train_tr Loss: 0.0224 Acc: 0.6422
val_tr Loss: 0.2246 Acc: 0.7728

Epoch 5/49
----------
train_tr Loss: 0.0169 Acc: 0.7183
val_tr Loss: 0.1659 Acc: 0.8400

Epoch 6/49
----------
train_tr Loss: 0.0135 Acc: 0.7681
val_tr Loss: 0.1558 Acc: 0.8408

Epoch 7/49
----------
train_tr Loss: 0.0110 Acc: 0.8085
val_tr Loss: 0.1312 Acc: 0.8787

Epoch 8/49
----------
train_tr Loss: 0.0089 Acc: 0.8399
val_tr Loss: 0.1188 Acc: 0.8825

Epoch 9/49
----------
train_tr Loss: 0.0075 Acc: 0.8657
val_tr Loss: 0.1063 Acc: 0.8957

Epoch 10/49
----------
train_tr Loss: 0.0061 Acc: 0.8852
val_tr Loss: 0.1017 Acc: 0.9019

Epoch 11/49
--------

In [ ]:
torch.save(model_conv, 'finetuned_resnet_market_all.pt')
#model = torch.load('finetuned_resnet_market_all.pt')

In [ ]:
#torch.save(model_conv, 'finetuned_resnet_market_fc_lr1s7g1_market_all.pt')
#model = torch.load('finetuned_resnet_market_fc_lr1s7g1_market_all.pt')

### eval

In [ ]:
class MarketDataset(Dataset):
    def __init__(self, df, le=None, check_junk_distractor=False, transform=None):
        self.transform = transform
        self.img_paths = np.array(df['pth'])
        
        if le is not None:
            self.has_classes = True
            self.classes = np.array(df['cl'].drop_duplicates())
            self.labels = torch.from_numpy(le.transform((df['cl'])))
        else:
            self.has_classes = False
            
        if check_junk_distractor:
            self.junk = df[df['cl'] == '-1']['pth']
            self.distractor = df[df['cl'] == '0000']['pth']

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, index):
        img_path = self.img_paths[index]
        img = Image.open(img_path)

        if self.transform:
            img = self.transform(img)

        if self.has_classes:
            cl = self.labels[index]
            return (img, cl)
        else:
            return img

In [ ]:
def create_df(x):
    df = pd.DataFrame({'pth': x})
    df['cl'] = df.pth.apply(lambda x: int(x.split('/')[-1].split('_')[0]))
    return df

In [ ]:
root = 'data/Market-1501-v15.09.15'
paths = {
    #'train': 'bounding_box_train/*.jpg',
    'test': 'bounding_box_test/*.jpg',
    'query': 'query/*.jpg'
}

dfs = {
    x: create_df(glob(os.path.join(root, paths[x])))for x in paths.keys()
}

for key in dfs.keys():
    print(key, len(dfs[key]))
    dfs[key].to_csv('%s.csv' % key, index=None, header=None)

In [ ]:
transform_dict = {
    'test': 'test',
    'query': 'test'
}

image_datasets = {
    x[0]: MarketDataset(dfs[x], le=x[1], check_junk_distractor=x[2] transform=data_transforms[transform_dict[x]]) 
    for x in [('test', le, True), ('query', None, True)]
}